In [0]:
# ============================================================
# Dimensão: ASSISTENTES (Bronze)
# Geração de hierarquia sintética (superintendentes→gerentes→coordenadores→supervisores→assistentes)
# Gravação como Tabela Managed em UC
# Padrão de namespace: <catalog>.<schema>.dim_assistentes
#
# Parâmetros esperados (dbutils.widgets):
#   - catalog (default: prd)
#   - schema  (default: b_dm_callcenter)
#   - table   (default: dim_assistentes)
#   - qtd_assistentes (default: 1000)
# ============================================================

from pyspark.sql import functions as F
import random, uuid
from faker import Faker

# ---------------- Parâmetros ----------------
try:
    catalog = dbutils.widgets.get("catalog")
except:
    dbutils.widgets.text("catalog", "prd")
    catalog = dbutils.widgets.get("catalog")

try:
    schema = dbutils.widgets.get("schema")
except:
    dbutils.widgets.text("schema", "b_dm_callcenter")
    schema = dbutils.widgets.get("schema")

try:
    table = dbutils.widgets.get("table")
except:
    dbutils.widgets.text("table", "dim_assistentes")
    table = dbutils.widgets.get("table")

try:
    qtd_assistentes = int(dbutils.widgets.get("qtd_assistentes"))
except:
    dbutils.widgets.text("qtd_assistentes", "1000")
    qtd_assistentes = int(dbutils.widgets.get("qtd_assistentes"))

fake = Faker("pt_BR")

# Seed opcional
# random.seed(42)
# Faker.seed(42)

# Tamanhos da hierarquia (ajuste conforme sua necessidade)
qtd_superintendentes = 2
qtd_gerentes = 4
qtd_coordenadores = 16
qtd_supervisores = 64

# ---------------- Geração de hierarquia ----------------
superintendentes = [{
    "id": str(uuid.uuid4()),
    "nome": fake.name()
} for _ in range(qtd_superintendentes)]

gerentes = [{
    "id": str(uuid.uuid4()),
    "nome": fake.name(),
    "id_superintendente": s["id"],
    "nome_superintendente": s["nome"]
} for s in random.choices(superintendentes, k=qtd_gerentes)]

coordenadores = [{
    "id": str(uuid.uuid4()),
    "nome": fake.name(),
    "id_gerente": g["id"],
    "nome_gerente": g["nome"],
    "id_superintendente": g["id_superintendente"],
    "nome_superintendente": g["nome_superintendente"]
} for g in random.choices(gerentes, k=qtd_coordenadores)]

supervisores = [{
    "id": str(uuid.uuid4()),
    "nome": fake.name(),
    "id_coordenador": c["id"],
    "nome_coordenador": c["nome"],
    "id_gerente": c["id_gerente"],
    "nome_gerente": c["nome_gerente"],
    "id_superintendente": c["id_superintendente"],
    "nome_superintendente": c["nome_superintendente"]
} for c in random.choices(coordenadores, k=qtd_supervisores)]

assistentes = []
for idx in range(1, qtd_assistentes + 1):
    supervisor = random.choice(supervisores)
    assistentes.append({
        "identificadorAssistente": idx,
        "nomeAssistente": fake.name(),
        "email": fake.email(),
        "matricula": f"MAT{random.randint(100000, 999999)}",
        "area": random.choice(["Atendimento", "Cobrança", "Financeiro", "Relacionamento"]),
        "identificadorSupervisor": supervisor["id"],
        "nomeSupervisor": supervisor["nome"],
        "identificadorCoordenador": supervisor["id_coordenador"],
        "nomeCoordenador": supervisor["nome_coordenador"],
        "identificadorGerente": supervisor["id_gerente"],
        "nomeGerente": supervisor["nome_gerente"],
        "identificadorSuperintendente": supervisor["id_superintendente"],
        "nomeSuperintendente": supervisor["nome_superintendente"],
        "status": random.choices(["ativo", "inativo"], weights=[0.95, 0.05])[0],
        "dt_cadastro": fake.date_time_this_year()
    })

df = spark.createDataFrame(assistentes) \
    .withColumn("dat_ref_carga", F.current_date()) \
    .withColumn("ingestion_ts", F.current_timestamp())

# ---------------- Escrita na Bronze (Managed Table) ----------------
db_table = f"{catalog}.{schema}.{table}"

(
    df.write
      .format("delta")
      .mode("overwrite")
      .option("overwriteSchema", "true")
      .saveAsTable(db_table)
)

spark.sql(f"OPTIMIZE {db_table}")

print(f"[OK] Dimensão criada/atualizada: {db_table}")
